In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [4]:
dataset = pd.read_csv('bank.csv', index_col = 'RowNumber')
dataset.head()

CustomerId   Surname  CreditScore Geography  Gender  Age  Tenure  \
RowNumber                                                                     
1            15634602  Hargrave          619    France  Female   42       2   
2            15647311      Hill          608     Spain  Female   41       1   
3            15619304      Onio          502    France  Female   42       8   
4            15701354      Boni          699    France  Female   39       1   
5            15737888  Mitchell          850     Spain  Female   43       2   

             Balance  NumOfProducts  HasCrCard  IsActiveMember  \
RowNumber                                                        
1               0.00              1          1               1   
2           83807.86              1          0               1   
3          159660.80              3          1               0   
4               0.00              2          0               0   
5          125510.82              1          1               1   

           EstimatedSalary  Exited  
RowNumber                           
1                101348.88       1  
2                112542.58       0  
3                113931.57       1  
4                 93826.63       0  
5                 79084.10       0

In [6]:
dataset.shape

(10000, 13)

In [7]:
X_columns = dataset.columns.tolist()[2:12]
Y_columns = dataset.columns.tolist()[-1:]
print(X_columns)
print(Y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [4]:
X = dataset[X_columns].values 
Y = dataset[Y_columns].values

In [5]:
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])

In [6]:
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ], 
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [8]:
X = pipeline.fit_transform(X)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input

# Initialize the classifier
classifier = Sequential()

# Add input layer and hidden layer
classifier.add(Dense(units=6, activation='relu', input_dim=X_train.shape[1]))
classifier.add(Dropout(rate=0.1))

In [11]:
#Add second layer
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [12]:
#Add output layer
classifier.add(Dense(1, activation = 'sigmoid'))

In [13]:
#Let us take a look at our network
classifier.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 6)                   │              72 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 6)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │              42 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 6)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               7 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 121 (484.00 B)

 Trainable params: 121 (484.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
#Optimize the weights
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [15]:
#Fitting the Neural Network
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split = 0.1, verbose = 2)

Epoch 1/200
225/225 - 3s - 11ms/step - accuracy: 0.7199 - loss: 0.5777 - val_accuracy: 0.7950 - val_loss: 0.4793
Epoch 2/200
225/225 - 1s - 5ms/step - accuracy: 0.7942 - loss: 0.4837 - val_accuracy: 0.7987 - val_loss: 0.4424
Epoch 3/200
225/225 - 1s - 3ms/step - accuracy: 0.7972 - loss: 0.4600 - val_accuracy: 0.8138 - val_loss: 0.4242
Epoch 4/200
225/225 - 0s - 2ms/step - accuracy: 0.8065 - loss: 0.4448 - val_accuracy: 0.8175 - val_loss: 0.4101
Epoch 5/200
225/225 - 1s - 3ms/step - accuracy: 0.8076 - loss: 0.4352 - val_accuracy: 0.8225 - val_loss: 0.3997
Epoch 6/200
225/225 - 0s - 2ms/step - accuracy: 0.8132 - loss: 0.4295 - val_accuracy: 0.8313 - val_loss: 0.3914
Epoch 7/200
225/225 - 0s - 2ms/step - accuracy: 0.8100 - loss: 0.4232 - val_accuracy: 0.8363 - val_loss: 0.3841
Epoch 8/200
225/225 - 1s - 3ms/step - accuracy: 0.8192 - loss: 0.4122 - val_accuracy: 0.8350 - val_loss: 0.3755
Epoch 9/200
225/225 - 1s - 2ms/step - accuracy: 0.8256 - loss: 0.4050 - val_accuracy: 0.8462 - val_loss

In [16]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
[[0.26894277]
 [0.31418166]
 [0.16971996]
 [0.05702815]
 [0.09412862]]


In [17]:
#Let us use confusion matrix with cutoff value as 0.5
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [18]:
#Making the Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1556   39]
 [ 235  170]]


In [19]:
#Accuracy of our NN
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

86.3 % of data was classified correctly
